In [2]:
import pandas as pd
import tester
from trainer import read_tables
import os
import argparse
import json
from IPython.display import display, clear_output
import time


In [3]:
def get_args(dataset):
    work_dir=os.path.abspath('..')
    test_args = argparse.Namespace(work_dir=work_dir,
                                   query_dir='demo_query',
                                   dataset=dataset)
    return test_args

In [4]:
def create_query_file(data_dir, question):
    file_name = 'fusion_query.jsonl'
    data_file = os.path.join(data_dir, file_name)
    query_data = {
        "id":0,
        "question":question,
        "table_id_lst":['N/A'],
        "answers":["N/A"],
        "ctxs": [{"title": "", "text": "This is a example passage."}]
    }
    with open(data_file, 'w') as f_o:
        f_o.write(json.dumps(query_data) + '\n')
    

In [5]:
def show_results(question, out_dir, table_dict):
    data_file = os.path.join(out_dir, 'pred_None.jsonl')
    with open(data_file) as f:
        item = json.load(f)
    tag_lst = item['tags']
    N = 5
    pred_table_set = set()
    for tag in tag_lst:
        table_id = tag['table_id']
        if table_id not in pred_table_set:
            pred_table_set.add(table_id)
            table = table_dict[table_id]
            show_table(question, table, len(pred_table_set))
        if len(pred_table_set) >= 5:
            return
    
def show_table(question, table, seq_no):
    print("Question, %s" % question)
    print("Top %d table,         %s " % (seq_no, table['documentTitle']))
    col_lst = table['columns']
    col_names = [a['text'] for a in col_lst]
    row_lst = table['rows']
    row_data = []
    for row_item in row_lst:
        cell_lst = row_item['cells']
        row_item = [a['text'] for a in cell_lst]
        row_data.append(row_item)
    df = pd.DataFrame(row_data,columns=col_names)
    display(df)
    

In [ ]:
dataset=input('Type the dataset, ')
args = get_args(dataset)
print('Loading tables')
table_dict = read_tables(args.work_dir, dataset)
data_dir = os.path.join(args.work_dir, 'data', dataset, 'demo_query', 'test')
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)
question = input('Type a question (q to quit),\n').strip()
while question != 'q':
    create_query_file(data_dir, question)
    out_dir = tester.main(args)
    clear_output(wait=False)
    show_results(question, out_dir, table_dict)
    time.sleep(1)
    question = input('Type a question (q to quit),\n').strip()
    

Question, What is the biggest city in US?
Top 1 table,         List of protests in the United States by size , List 


,-,Protest,City,Estimated participants,Date,Image
0,1,2017 Women's March,—,"3,300,000–4,600,000","January 21, 2017",-
1,2,2018 Women's March,—,"1,500,000","January 20, 2018",2018 Women's March in New York City
2,3,March for Our Lives,—,"1,200,000-2,000,000","March 24, 2018","Protesters in Washington, D.C., 2018"
3,4,March for Science,—,"~1,000,000","April 22, 2017","Protesters in Washington, D.C., 2017"
4,5,"March on Washington for Lesbian, Gay and Bi Eq...","Washington, D.C.","800,000–1,000,000","April 25, 1993",Demonstrators in front of the White House
5,6,"Anti-nuclear weapon march, part of the Nuclear...",New York City,"700,000–1,000,000","June 12, 1982",-
6,7,Million Man March,"Washington, D.C.","670,000–800,000","October 16, 1995",-
7,8,March for Women's Lives,"Washington, D.C.","500,000–1,000,000","April 25, 2004",-
8,9,Million Mom March,"Washington, D.C.","750,000","May 14, 2000",-
9,10,March for Life,"Washington, D.C.","400,000-650,000 (2013 estimate from rally orga...","Annually since January 22, 1974",2013 march


Question, What is the biggest city in US?
Top 2 table,         List of largest employers , Largest employers 


,-,Employees,Employees,-
0,Employer,2015,2010,Headquarters
1,United States Department of Defense,2.86 million,2.86 million,United States
2,People's Liberation Army,2.52 million,2.52 million,China
3,Walmart,2.3 million,2.3 million,United States
4,McDonald's,1.9 million,1.7 million,United States
5,National Health Service,1.7 million,1.4 million,United Kingdom
6,China National Petroleum Corporation,1.6 million,1.7 million,China
7,State Grid Corporation of China,1.5 million,1.6 million,China
8,Indian Railways,1.4 million,1.3 million,India
9,Indian Armed Forces,1.3 million,1.3 million,India


Question, What is the biggest city in US?
Top 3 table,         Gay village , By U.S. city 


,Rank,City,Percentage of City Population,LGBT Population,LGBT Population
0,Rank,City,Percentage of City Population,population,rank
1,1,New York City,4.5%,"272,493",1
2,2,Los Angeles,5.6%,"154,270",2
3,3,Chicago,5.7%,"114,449",3
4,4,San Francisco,15.4%,"94,234",4
5,5,Phoenix,6.4%,"63,222",5
6,6,Houston,4.4%,"61,976",6
7,7,San Diego,6.8%,"61,945",7
8,8,Dallas,7.0%,"58,473",8
9,9,Seattle,12.9%,"57,993",9


Question, What is the biggest city in US?
Top 4 table,         List of longest-serving mayors in the United States , Current and former mayors 


,Years,Name,Municipality,Notes
0,"63 years, 73 days",Hilmar Moore,"Richmond, Texas","Served from September 22, 1949 to December 4, ..."
1,"61 years, 111 days",John H. Land,"Apopka, Florida","Served from January 1, 1971 to April 22, 2014...."
2,"58 years, 233 days",Robert Linn,"Beaver, Pennsylvania","Served from January 2, 1946 to August 22, 2004."
3,59–60 years,Charles E. Long,"Booneville, Kentucky",Entered office in 1959.
4,56 years,Paul Jurko,"Yankee Lake, Ohio",Elected into office in November 1931 at the ag...
5,54 years,Robert L. Butler,"Marion, Illinois",Entered office in May 1963. Served until his r...
6,"52 years, 0 days",John M. Coyne,"Brooklyn, Ohio",Served from 1948 until January 2000.
7,"55 years, 92 days",Bruce Arnold,"Valparaiso, Florida","Entered office on January 21, 1964, Died in of..."
8,51 years,Donald Stephens,"Rosemont, Illinois",First Mayor of Rosemont. Served from village's...
9,50 years,Leonard T. Connors,"Surf City, New Jersey","Entered office in 1966, served until January 5..."


Question, What is the biggest city in US?
Top 5 table,         List of newspapers in the United States , Top 10 newspapers by circulation 


,Rank,Newspaper,Primary locality,Headquarters state or district,Total average circulation,Owner,Nameplate
0,1,USA Today,"McLean, Virginia",Virginia,"2,301,917",Gannett Company,-
1,2,The New York Times,New York City,New York,"2,101,611",The New York Times Company,-
2,3,The Wall Street Journal,New York City,New York,"1,321,827",News Corp,-
3,4,Los Angeles Times,Los Angeles,California,"467,309",Nant Capital,-
4,5,New York Post,New York,New York,"424,721",News Corp,-
5,6,Chicago Tribune,Chicago,Illinois,"384,962",Tribune Publishing,-
6,7,The Washington Post,Washington D.C.,District of Columbia,"356,768",Nash Holdings,-
7,8,Newsday,Melville,New York,"321,296",Newsday Media,-
8,9,Daily News,New York City,New York,"299,538",Tribune Publishing,-
9,10,AM New York,New York City,New York,"298,759",Newsday Media,-
